In [1]:
import pandas as pd
import cleaningtools as ct 
import file_config as fconfig
import merge_config as config
import assay_sample_config as aconfig
import spectral_config as sconfig
import xrf_config as xconfig
import numpy as np
import math
from tqdm.notebook import tqdm_notebook as tqdm
tqdm.pandas()
from tqdm import tqdm
import os
import file_config as fconfig
import pandas as pd

check for pandas
pandas is properly installed
check for tqdm
tqdm is properly installed
check for numpy
numpy is properly installed
check for openpyxl
openpyxl is properly installed


In [2]:
path='/Volumes/GoogleDrive/Shared drives/AMC Projects/_AZ_Kay/_Drilling/_Logs'
path_list=path.split('/')
b=path_list.index('_AZ_Kay')
base_path='/'.join(path_list[:b+1])

base_path


'/Volumes/GoogleDrive/Shared drives/AMC Projects/_AZ_Kay'

In [3]:
master_path=base_path+'/_Master Databases/spectral master.csv'
master_path


'/Volumes/GoogleDrive/Shared drives/AMC Projects/_AZ_Kay/_Master Databases/spectral master.csv'

In [4]:

data_list=[]
i=0
for folder in os.listdir(path):
    folder=path+'/'+folder
    if os.path.isdir(folder):
        for file in os.listdir(folder):
                if 'hyp-pkg samples' in file.lower():
                    print(i,file)
                    name=file
                    file=folder+'/'+file
                    try:
                        data=pd.read_excel(file, header=None,na_filter=True)
                        data=data.dropna(how='all')
                        headers = data.iloc[0].str.lower()
                        data  = pd.DataFrame(data.values[1:], columns=headers).reset_index(drop=True)
                        if data.T. index.duplicated().any():
                            print(i,'duplcate columns')
                            continue
                        data['FileName']=name
                        data_list.append(data)
                        i+=1
                    except Exception as e:
                        print(e)
    else: 
        print(f"{folder} is not a folder")

0 KM-22-81  HYP-PKG Samples.xlsx
1 KM-22-62  HYP-PKG Samples.xlsx
2 KM-22-62A  HYP-PKG Samples.xlsx
3 KM-22-62B  HYP-PKG Samples.xlsx
4 KM-22-57C  HYP-PKG Samples.xlsx
5 KM-22-60  HYP-PKG Samples.xlsx
6 KM-22-59A  HYP-PKG Samples.xlsx
7 KM-22-61  HYP-PKG Samples.xlsx
8 KM-22-62C  HYP-PKG Samples.xlsx
9 KM-22-63  HYP-PKG Samples.xlsx
10 KM-22-63A  HYP-PKG Samples.xlsx
11 KM-22-64  HYP-PKG Samples.xlsx
12 KM-22-67  HYP-PKG Samples.xlsx
13 KM-22-65  HYP-PKG Samples.xlsx
14 KM-22-71 HYP-PKG Samples.xlsx
15 KM-22-69  HYP-PKG Samples.xlsx
16 KM-22-68  HYP-PKG Samples.xlsx
17 KM-22-66  HYP-PKG Samples.xlsx
18 KM-22-57B  HYP-PKG Samples.xlsx
19 KM-22-71A  HYP-PKG Samples.xlsx
20 KM-22-72  HYP-PKG Samples.xlsx
21 KM-22-63B HYP-PKG Samples.xlsx
22 KM-22-73  HYP-PKG Samples.xlsx
23 KM-22-74  HYP-PKG Samples.xlsx
24 KM-22-63C  HYP-PKG Samples.xlsx
25 KM-22-75  HYP-PKG Samples.xlsx
26 KM-22-XX  HYP-PKG Samples.xlsx
26 duplcate columns
26  KM-22-76  HYP-PKG Samples.xlsx
/Volumes/GoogleDrive/Shared d

In [5]:
spec_samples=pd.concat(data_list,axis=0,ignore_index=True)
spec_samples['from_ft']=spec_samples['depth_ft']
for hole in spec_samples.hole_id.unique():
    spec_samples.loc[spec_samples.hole_id==hole,'from_ft']=pd.to_numeric(spec_samples.loc[spec_samples.hole_id==hole,'depth_ft'].values,errors='coerce')
    spec_samples.loc[spec_samples.hole_id==hole,'to_ft']=spec_samples.loc[spec_samples.hole_id==hole,'from_ft'].shift(-1).values-.5
    spec_samples.loc[spec_samples.hole_id==hole,'from_m']=spec_samples.loc[spec_samples.hole_id==hole,'from_ft'].values*0.3048
    spec_samples.loc[spec_samples.hole_id==hole,'to_m']=spec_samples.loc[spec_samples.hole_id==hole,'to_ft'].values*0.3048
spec_samples=spec_samples.drop(spec_samples.filter(like='depth'),axis=1)

spec_samples=ct.remove_depth_errors(spec_samples)

In [6]:
spectral_master=pd.read_csv('/Volumes/GoogleDrive/Shared drives/AMC Projects/_AZ_Kay/_Master Databases/spectral master.csv')
spectral_master

/var/folders/jc/lq6pqgbn3dlfs9v5g5x4jc_80000gn/T/ipykernel_53249/2935086753.py:1: DtypeWarning: Columns (94,95,98,99,100,151,155,156,157,158,159) have mixed types. Specify dtype option on import or set low_memory=False.
  spectral_master=pd.read_csv('/Volumes/GoogleDrive/Shared drives/AMC Projects/_AZ_Kay/_Master Databases/spectral master.csv')


,Spectrum,FileName,ProjectPath,Company,Country,Region,Project,OperatorName,DispatchID,LabJobNumber,...,filename,SAMPLE,Recvd Wt.,START,END,SpectrumId,Chlorite_vnir,Amphibole_vnir,WhiteMica_SC\r,Tourmaline_vnir
0,Spectrum001.asd,Spectrum001.asd,Arizona Metals Corp.USA..Kay Mine,Arizona Metals Corp,USA,NaN,Kay Mine,NaN,,TU22096305,...,TU22096305_INTERP.csv,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Spectrum002.asd,Spectrum002.asd,Arizona Metals Corp.USA..Kay Mine,Arizona Metals Corp,USA,NaN,Kay Mine,NaN,,TU22096305,...,TU22096305_INTERP.csv,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Spectrum003.asd,Spectrum003.asd,Arizona Metals Corp.USA..Kay Mine,Arizona Metals Corp,USA,NaN,Kay Mine,NaN,,TU22096305,...,TU22096305_INTERP.csv,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Spectrum004.asd,Spectrum004.asd,Arizona Metals Corp.USA..Kay Mine,Arizona Metals Corp,USA,NaN,Kay Mine,NaN,,TU22096305,...,TU22096305_INTERP.csv,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Spectrum005.asd,Spectrum005.asd,Arizona Metals Corp.USA..Kay Mine,Arizona Metals Corp,USA,NaN,Kay Mine,NaN,,TU22096305,...,TU22096305_INTERP.csv,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8039,Available,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,TU20133360.csv,B300331,NaN,73.0,74.0,Available,Present,NaN,NaN,NaN
8040,Available,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,TU20133360.csv,B300332,NaN,74.0,75.0,Available,NaN,NaN,NaN,NaN
8041,Available,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,TU20133360.csv,B300333,NaN,75.0,76.0,Available,NaN,NaN,NaN,NaN
8042,Available,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,TU20133360.csv,B300334,NaN,76.0,77.0,Available,Present,NaN,NaN,NaN


In [7]:
spec=spectral_master.merge(spec_samples,how='outer',left_on='sample_id',right_on='sample_id')
spec

KeyError: 'sample_id'

In [ ]:
starting_cols=['sample_id','from_ft','to_ft','from_m','to_m','hole_id','geo']
ending_cols=[col for col in spec.columns if col not in starting_cols]
spec=pd.concat([spec[starting_cols],spec[ending_cols]],axis=1)
spec

In [ ]:
spec.to_csv(master_path,index=False)